# Capítulo 1 - O Problema

Temos no CEFET MG um grupo chamado Grupo de Computação Competitiva (GCC) que tem como uma de suas atividades a participação na maratona de programação.

A maratona de programação é uma competição onde equipes competem entre si em um período de tempo pré determinado a tentar resolver o maior número de problema em menos tempo.

O objetivo deste trabalho, é desenvolver um algoritmo de recomendação que ajude no treinamento dos competidores para a preparação para a maratona de programação.

Atualmente, o treinamento funciona da seguinte maneira. Existem alguns sites, chamados de juízes online, que funcionam como um repositório de problemas. O competidor então consegue ler o problema, desenvolver uma solução e submete-la para o juiz. O juiz então dará a resposta se a solução foi aceita ou não.

O problema do treinamento atual é que existem milhares de problemas a serem resolvidos. Isso faz com que o competidor perca tempo em problemas que não acrescentem em conteúdo para o seu desenvolvimento, ou então, escolha problemas muito difíceis para serem resolvidos, o que torna seu treinamento desmotivador.


# Capítulo 2 - Dataset


Foram coletados do site URI Online Judge, mais de 200 mil soluções de mais de 600 usuários. Cada solução é composta por:

 - **user_id**: usuário que resolveu o problema
 - **problem_id**: problema que foi resolvido
 - **date**: data de quando o problema foi resolvido
 - **category_id**: categoria do problema resolvido
 - **level**: dificuldade do problema resolvido
 - **solved**: quantidade total de usuários do juiz que resolveram o problema
 - **name**: nome do problema
 - **topics**: assuntos relacionados ao conteúdo do problema

In [32]:
import pandas as pd
import numpy as np
from numpy.random import rand
from scipy import spatial
from datetime import datetime
from pprint import pprint


solutions_df = pd.read_csv('solutions.csv')
solutions_df[:5]

,user_id,problem_id,date,category_id,level,solved,name,topics
0,40980,1001,15/02/25 06:15,1,1,127331,Extremely Basic,sequential
1,40980,1002,15/02/25 07:10,1,1,87459,Area of a Circle,sequential
2,40980,1003,15/02/25 06:17,1,1,89856,Simple Sum,sequential
3,40980,1004,15/02/25 06:19,1,1,85938,Simple Product,sequential
4,40980,1005,15/02/25 07:31,1,1,73427,Average 1,sequential


# Capítulo 3 - Filtragem dos dados

De forma a otimizar o tempo de desenvolvimento, é possível realizar um filtro sobre as categorias disponíveis.

In [2]:
filter_category = 7

categories = [
    'All', #0
    'Beginner', #1
    'Ad-Hoc', #2
    'String', #3
    'Libraries', #4
    'Math', #5
    'Paradigms', #6
    'Graph', #7
    'Geometry', #8
]

category = categories[filter_category]

if filter_category:
    solutions_df = solutions_df.loc[solutions_df['category_id'] == filter_category]

Quantidade de soluções do dataset:

In [3]:
len(solutions_df)

11403

Quantidade de problemas do dataset:

In [4]:
solutions_df['problem_id'].nunique()

181

Quantidade de usuários do dataset:

In [5]:
solutions_df['user_id'].nunique()

558

Quantidade de categorias do dataset:

In [6]:
solutions_df['category_id'].nunique()

1

# Capítulo 4 - Desenvolvimento

Uma série de técnicas de Machine Learning serão utilizadas a fim de obter o resultado final. Serão aqui descritas passo a passo, os procedimentos adotados.

obs: cada exemplo poderia conter o link de um problema

- storytelling

Em todo período acadêmico, o conteúdo aprendido é desenvolvido de forma gradual. Tal como as aulas de cálculo, onde começamos pelo o estudo de funções, passamos por limites e derivadas até finalmente o estudo das integrais.
Desde a primeira aula, todo o conteúdo aprendido, é fundamental para o desenvolvimento dos demais.

Da mesma forma funciona o treinamento para a maratona de programação. Por exemplo, no estudo de estruturas de dados, iniciamos com o estudo de estruturas de dados simples como pilhas e filas, passando por estruturas um pouco mais complexas como árvores e grafos até chegar em algoritmos que utilizam tais estruturas de dados mais complexas como a Segment Tree.

Baseado nas informações acima citadas, podemos concluir que um competidor para resolver um problema mais complexo, necessita desenvolver conhecimentos fundamentais. E a partir desta conclusão temos o primeiro problema a ser resolvido por este trabalho: **Identificar os tópicos necessários a se saber para a solução de um problema**

Tendo isso em mente, essa será a primeira tarefa desenvolvida. Para isso iremos utilizar a técnica de Collaborative Filtering.

Seja p um problema e u um usuario do nosso dataset. Serão filtradas todas as soluções submetidas pelo usuário u antes da solução do problema p. Isso nos permitirá identificar quais os tópicos que foram desenvolvidos pelo usuário u, através das soluções dos problemas anteriores, que o capacitaram a solucionar o problema p.



## Colaborative Filtering

> A Colaborative Filtering é um método para fazer previsões automáticas (filtragem) sobre os interesses de um usuário, coletando preferências ou informações de gosto de muitos usuários (colaborando). O pressuposto subjacente da abordagem de filtragem colaborativa é que se uma pessoa A tiver a mesma opinião que uma pessoa B em um problema, é mais provável que a opinião de B seja diferente em relação a uma pessoa escolhida aleatoriamente. Por exemplo, um sistema de recomendação de filtragem colaborativa para os gostos da televisão poderia fazer previsões sobre o programa de televisão que um usuário deveria gostar, dado uma lista parcial dos gostos desse usuário (gosta ou não gosta).

Imagine uma situação onde temos um dataset composto por avaliações de filmes Cada avaliação é feita através de uma nota entre 1 e 5. Teriamos uma tabela da seguinte forma:

| Filmes  | Ana | Beto | Carlos | Dani |
|---------|:---:|:----:|:------:|:----:|
| Querido John |  5  |   4  |    0   |   0  |
| Titanic |  3  |   5  |    0   |   0  |
| Batman |  0  |   0  |    4   |   5  |
| Thor |  0  |   0  |    5   |   4  |

As notas dadas aos filmes podem ser relacionadas as preferências do usuário em relação aos gêneros dos filmes. A alta nota de Beto ao filme Titanic pode ser justificada por Beto gostar de filmes de romance.

Logo, se fossem conhecidos as caraterísticas dos filmes e as preferências do usuários, poderiamos prever as notas dos filmes que ainda não foram assistidos pelo o usuário.


| Filmes  | Ana | Beto | Carlos | Dani | Ação | Romance |
|---------|:--:|:--:|:--:|:--:|:--:|:--:|
| Querido John | 5 | 5 | 0 | 0 | ? | ? |
| Titanic | 5 | 5 | 0 | 0 | ? | ? |
| Batman | 0 | 0 | 5 | 5 | ? | ? |
| Thor | 0 | 0 | 5 | 5 | ? | ? |

Podemos então representar cada filme com um vetor de **características** $X$ e cada usuário com um vetor de **preferências** $\theta$ ambos de mesma dimensão. A nota final do filme, pode ser então dada pelo produto $X ⋅ \theta^T$

Embora não tenhamos os valores de $X$ e $\theta$ estes valores podem ser obtidos através do nosso dataset da seguinte forma.

Os valores de $X$ serão iniciados com valores randômicos. A partir daí temos um problema de otimização, onde o nosso objetivo é minizar a diferença entre a nota do filme $f$ dado pelo usuário $u$ pelo produto $ X_u \cdot \theta_f^T $.

Iremos então realizar o mesmo procedimento, porém agora iremos encontrar os valores de $X$ utilizando os valores de $\theta$ encontrados. 


Este procedimento pode ser então repetido até encontrarmos valores estáveis de $\theta$ e $X$ ou por um número finito de iterações.

In [34]:
# http://www.bogotobogo.com/python/python_numpy_batch_gradient_descent_algorithm.php
def gradient_descent(x, y, alpha=0.01, nr_iterations=1000):

    # m - número de amostras
    # n - número features
    m, n = x.shape

    # Insere coluna theta zero = 1
    x = np.c_[ np.ones(m), x]

    theta = np.ones(n + 1)
    x_transpose = x.transpose()

    for iter in range(0, nr_iterations):
        hypothesis = np.dot(x, theta)
        loss = hypothesis - y
        J = np.sum(loss ** 2) / (2 * m)
        gradient = np.dot(x_transpose, loss) / m         
        theta = theta - alpha * gradient

    return theta

theta_usuarios = [
    [rand(), rand()],
    [rand(), rand()],
    [rand(), rand()],
    [rand(), rand()],
]

theta_filmes = [
    [None, None],
    [None, None],
    [None, None],
    [None, None],
]

ratings = [
    [5, 4, 0, 0],
    [3, 5, 0, 0],
    [0, 0, 4, 5],
    [0, 0, 5, 4],
]

ratings_t = np.transpose(ratings).tolist()

nr_usuarios = len(theta_usuarios)
nr_filmes = len(theta_filmes)

i = 0
while i < 1000:
    i += 1

    for filme in range(nr_filmes):
        x = np.array(
            theta_usuarios
        )

        y = np.array(
            ratings[filme]
        )

        theta_filmes[filme] = gradient_descent(x, y)[1:]


    for usuario in range(nr_usuarios):
        x = np.array(
            theta_filmes
        )

        y = np.array(
            ratings_t[usuario]
        )

        theta_usuarios[usuario] = gradient_descent(x, y)[1:]

print('Theta Usuarios')
pprint(theta_usuarios)
print('')
print('Theta Filmes')
pprint(theta_filmes)

Theta Usuarios
[array([ 0.92465498, -1.04562627]),
 array([-0.7960957 , -0.93829834]),
 array([ 0.26694584,  1.007224  ]),
 array([ 0.26694584,  1.007224  ])]

Theta Filmes
[array([ 0.45031437, -2.29674282]),
 array([-1.22013818, -1.87818132]),
 array([ 0.17404979,  2.23267427]),
 array([ 0.17404979,  2.23267427])]


In [41]:
np.dot(theta_usuarios, np.transpose(theta_filmes))

array([[ 2.81792004,  0.83566887, -2.17360686, -2.17360686],
       [ 1.79653664,  2.73364116, -2.23347485, -2.23347485],
       [-2.19312495, -2.21746013,  2.29526499,  2.29526499],
       [-2.19312495, -2.21746013,  2.29526499,  2.29526499]])

Agora podemos utilizar a mesma idéia no nosso problema.

Primeiramente iremos identificar as características de cada problema. Iremos então identificar as características do usuário, e assim então poderemos encontrar o problema mais adequado ao treinamento do usuário.


As características do problema.

Partindo do pressuposto 

In [29]:
def get_solution_date(solutions_df, user_id, problem_id):
    solution = solutions_df[(solutions_df['problem_id'] == problem_id) & (solutions_df['user_id'] == user_id)]
    solution_date = solution['date'].values[0]

    return solution_date


def get_past_problems(solutions_df, user_id, problem_id):
    solution_date = get_solution_date(solutions_df, user_id, problem_id)
    past_solutions = solutions_df[(solutions_df['date'] < solution_date) & (solutions_df['user_id'] == user_id)]
    problems = past_solutions['problem_id']

    return list(problems)


def get_users_that_solved_the_problem(solutions_df, problem_id):
    solutions_of_the_problem_df = solutions_df[solutions_df['problem_id'] == problem_id]
    users = solutions_of_the_problem_df['user_id']
    
    return list(users)


def get_problems(solutions_df):
    problems = set(solutions_df['problem_id'])

    return list(problems)


def get_solution_matrix(solutions_df, problem):
    users = get_users_that_solved_the_problem(solutions_df, problem)

    solutions = []
    min_problem = 9999
    max_problem = 0

    for user in users:
        past_problems = get_past_problems(solutions_df, user, problem)

        solutions.append(past_problems)

        min_problem = min([min_problem] + past_problems)
        max_problem = max([max_problem] + past_problems)

    nr_problems = max_problem - min_problem + 1
    nr_users = len(users)
    matrix = np.zeros((nr_problems, nr_users))

    for user in range(nr_users):
        for problem in solutions[user]:
            problem_index = problem - min_problem
            matrix[problem_index][user] = 1 # ou level do problema
            
    return matrix

def optimize_matrix(matrix):
    matrix = matrix[~np.all(matrix == 0., axis=1)]
    
    return matrix


# http://www.bogotobogo.com/python/python_numpy_batch_gradient_descent_algorithm.php
def gradient_descent(x, y, alpha=0.01, nr_iterations=100):

    # m - número de amostras
    # n - número features
    m, n = x.shape

    # Insere coluna theta zero = 1
    x = np.c_[ np.ones(m), x]

    theta = np.ones(n + 1)
    x_transpose = x.transpose()

    for iter in range(0, nr_iterations):
        hypothesis = np.dot(x, theta)
        loss = hypothesis - y
        J = np.sum(loss ** 2) / (2 * m)
        gradient = np.dot(x_transpose, loss) / m         
        theta = theta - alpha * gradient

    return theta


NR_FEATURES = 4

def get_theta_of_problem(matrix):
    nr_problems, nr_users = matrix.shape

    theta_users = np.random.rand(nr_users, NR_FEATURES)
    theta_problems = np.zeros((nr_problems, NR_FEATURES))


    # matrix
    #     u1  u2  u3
    # p1   1
    # p2   1  1    1
    # p3      1

    matrix_t = np.transpose(matrix)

    i = 0
    while i < 1:
        i += 1

        for problem in range(nr_problems):
            x = np.array(
                theta_users
            )

            y = np.array(
                matrix[problem]
            )

            theta_problems[problem] = gradient_descent(x, y)[1:]

        for user in range(nr_users):
            x = np.array(
                theta_problems
            )

            y = np.array(
                matrix_t[user]
            )

            theta_users[user] = gradient_descent(x, y)[1:]
    
    return theta_problems.mean(axis=0)


def get_theta_of_user(problems_solved, thetas_solved):
    
    if not problems_solved:
        return np.zeros(NR_FEATURES)

    nr_problems = len(problems_solved)

    y = np.ones(nr_problems)
    x = np.array([theta for theta in thetas_solved.values()])

    theta_of_user = gradient_descent(x, y)[1:]

    return theta_of_user    

def find_closest_problem(theta_user, theta_problems):
    
    problems = [problem for problem in theta_problems]
    points = [theta_problems[problem] for problem in theta_problems]
    tree = spatial.KDTree(points)
    
    distance, index = tree.query(theta_user)
    
    return problems[index]
    
def get_next_problem(problems_solved, thetas):
    thetas_solved = {}
    thetas_not_solved = {}
    
    for problem in problems_solved:
        thetas_solved[problem] = thetas[problem]
    
    for theta in thetas:
        if theta not in problems_solved:
            thetas_not_solved[theta] = thetas[theta]
            
    if not thetas_not_solved:
        return None
    
    theta_user = get_theta_of_user(problems_solved, thetas_solved)
    problem = find_closest_problem(theta_user, thetas_not_solved)
    
    return problem


In [8]:
thetas = {}

for problem in get_problems(solutions_df):
    matrix = get_solution_matrix(solutions_df, problem)
    matrix = optimize_matrix(matrix)
    theta = get_theta_of_problem(matrix)
    thetas[problem] = theta

nr_problems 151
nr_users 13
nr_problems 157
nr_users 96
nr_problems 160
nr_users 14
nr_problems 154
nr_users 228
nr_problems 156
nr_users 202
nr_problems 132
nr_users 7
nr_problems 165
nr_users 13
nr_problems 141
nr_users 11
nr_problems 159
nr_users 15
nr_problems 141
nr_users 102
nr_problems 154
nr_users 59
nr_problems 167
nr_users 18
nr_problems 164
nr_users 26
nr_problems 158
nr_users 14
nr_problems 160
nr_users 34
nr_problems 165
nr_users 51
nr_problems 156
nr_users 48
nr_problems 154
nr_users 6
nr_problems 137
nr_users 198
nr_problems 148
nr_users 10
nr_problems 150
nr_users 51
nr_problems 145
nr_users 312
nr_problems 135
nr_users 343
nr_problems 145
nr_users 108
nr_problems 121
nr_users 1
nr_problems 157
nr_users 19
nr_problems 167
nr_users 19
nr_problems 152
nr_users 93
nr_problems 141
nr_users 251
nr_problems 172
nr_users 49
nr_problems 149
nr_users 5
nr_problems 154
nr_users 29
nr_problems 166
nr_users 28
nr_problems 170
nr_users 62
nr_problems 167
nr_users 104
nr_problems 155

In [11]:
problems_solved = []
while True:
    next_problem = get_next_problem(problems_solved, thetas)
    
    if not next_problem:
        break
    
    problems_solved.append(next_problem)
    print(next_problem)

1502
1723
1752
1915
2117
1461
1954
2103
1742
2357
2073
1482
1747
2429
1951
2112
2359
1821
1413
2079
2048
1479
2082
1724
1695
2098
2039
1053
2002
1653
2184
1711
1823
1675
2182
2155
2127
1499
2088
2118
2131
1993
1207
1489
1698
1749
1402
1925
1415
1677
1979
1988
1617
2173
2490
2056
1328
1302
1442
1462
2130
1562
1592
1669
2190
1778
1123
1409
1205
1610
1764
1584
2404
1391
1490
2085
2081
2032
1928
1910
2419
1671
1423
2225
1389
1952
1813
1799
1227
1706
1790
1057
1194
2300
2476
1111
1862
2440
1525
1348
1738
2086
1506
1774
1270
1317
1344
1529
1792
1208
1135
1974
1757
1955
1956
1733
2485
1085
1447
1908
1977
1417
1330
1298
1907
1923
1855
1773
1835
1972
1454
1322
1702
1655
1713
2046
1152
1850
1621
2477
1314
1334
1931
1806
1362
1539
1498
1994
1076
1628
1384
1903
1782
1552
1081
1201
1427
1195
1476
1902
1128
1948
1200
1463
1550
1056
1469
1082
1751
1191
1466
1148
1692
1883
1100
1275
1668
2128
1894
1394
1826
